# Create the Blockchain class

In [3]:
import hashlib
from time import time
import json
from uuid import uuid4

class Blockchain(object):
    def __init__(self):
        self.current_transactions = []
        self.chain = []

        # Create the genesis block
        self.new_block(previous_hash=1, proof=100)

    def new_block(self, proof, previous_hash=None):
        """
        Create a new Block in the Blockchain
        :param proof: <int> The proof given by the Proof of Work algorithm
        :param previous_hash: (Optional) <str> Hash of previous Block
        :return: <dict> New Block
        """

        block = {
            'index': len(self.chain) + 1,
            'timestamp': time(),
            'transactions': self.current_transactions,
            'proof': proof,
            'previous_hash': previous_hash or self.hash(self.chain[-1]),
        }

        # Reset the current list of transactions
        self.current_transactions = []

        self.chain.append(block)
        return block

    def new_transaction(self, sender, recipient, amount):
        """
        Creates a new transaction to go into the next mined Block
        :param sender: <str> Address of the Sender
        :param recipient: <str> Address of the Recipient
        :param amount: <int> Amount
        :return: <int> The index of the Block that will hold this transaction
        """
        self.current_transactions.append({
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
        })

        return self.last_block()['index'] + 1


    def last_block(self):
        return self.chain[-1]


    def hash(self, block):
        """
        Creates a SHA-256 hash of a Block
        :param block: <dict> Block
        :return: <str>
        """

        # We must make sure that the Dictionary is Ordered, or we'll have inconsistent hashes
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()
    
    def proof_of_work(self, last_proof):
        """
        Simple Proof of Work Algorithm:
         - Find a number p' such that hash(pp') contains leading 4 zeroes, where p is the previous p'
         - p is the previous proof, and p' is the new proof
        :param last_proof: <int>
        :return: <int>
        """

        proof = 0
        while self.valid_proof(last_proof, proof) is False:
            proof += 1

        return proof

    @staticmethod
    def valid_proof(last_proof, proof):
        """
        Validates the Proof: Does hash(last_proof, proof) contain 4 leading zeroes?
        :param last_proof: <int> Previous Proof
        :param proof: <int> Current Proof
        :return: <bool> True if correct, False if not.
        """

        guess = f'{last_proof}{proof}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == "0000"
    
    

# Instantiate a blockchain and mine the first block 
(beyond the Genesis block which is mined a instantiation of the blockchain)

In [4]:
blockchain = Blockchain()

# universally unique identifier (UUID) is a 128-bit number used to identify information in computer systems
node_identifier = str(uuid4()).replace('-', '')

def mine():
    # We run the proof of work algorithm to get the next proof...
    last_block = blockchain.last_block()
    last_proof = last_block['proof']
    proof = blockchain.proof_of_work(last_proof)

    # We must receive a reward for finding the proof.
    # The sender is "0" to signify that this node has mined a new coin.
    blockchain.new_transaction(
        sender="0",
        recipient=node_identifier,
        amount=1,
    )

    # Forge the new Block by adding it to the chain
    previous_hash = blockchain.hash(last_block)
    block = blockchain.new_block(proof, previous_hash)

    response = {
        'message': "New Block Forged",
        'index': block['index'],
        'transactions': block['transactions'],
        'proof': block['proof'],
        'previous_hash': block['previous_hash'],
    }
    return response

mine()

{'index': 2,
 'message': 'New Block Forged',
 'previous_hash': 'a782f8423ac42166c7400f67c7c10664a8ca7497a90eb3206497d3ecbf0f0ad5',
 'proof': 35293,
 'transactions': [{'amount': 1,
   'recipient': '05069c3d8d3b490290ac0dbfb0d51b30',
   'sender': '0'}]}

# The block chain now looks like:

In [5]:
blockchain.chain

[{'index': 1,
  'previous_hash': 1,
  'proof': 100,
  'timestamp': 1516578139.736879,
  'transactions': []},
 {'index': 2,
  'previous_hash': 'a782f8423ac42166c7400f67c7c10664a8ca7497a90eb3206497d3ecbf0f0ad5',
  'proof': 35293,
  'timestamp': 1516578139.955023,
  'transactions': [{'amount': 1,
    'recipient': '05069c3d8d3b490290ac0dbfb0d51b30',
    'sender': '0'}]}]

# Let's add a transaction to the next block

In [6]:
blockchain.new_transaction("bob", "alice", "10")

3

In [7]:
blockchain.chain

[{'index': 1,
  'previous_hash': 1,
  'proof': 100,
  'timestamp': 1516578139.736879,
  'transactions': []},
 {'index': 2,
  'previous_hash': 'a782f8423ac42166c7400f67c7c10664a8ca7497a90eb3206497d3ecbf0f0ad5',
  'proof': 35293,
  'timestamp': 1516578139.955023,
  'transactions': [{'amount': 1,
    'recipient': '05069c3d8d3b490290ac0dbfb0d51b30',
    'sender': '0'}]}]

It doesn't appear because a new block hasn't been mined yet. It lives in blockchain.current_transactions list at the moment

In [9]:
blockchain.current_transactions

[{'amount': '10', 'recipient': 'alice', 'sender': 'bob'}]

Let's add come more

In [10]:
blockchain.new_transaction("tom", "brady", "1000000")
blockchain.new_transaction("john", "smith", "30")

3

In [11]:
blockchain.current_transactions

[{'amount': '10', 'recipient': 'alice', 'sender': 'bob'},
 {'amount': '1000000', 'recipient': 'brady', 'sender': 'tom'},
 {'amount': '30', 'recipient': 'smith', 'sender': 'john'}]

# Now, let's mine the next block

In [14]:
mine()
blockchain.chain

[{'index': 1,
  'previous_hash': 1,
  'proof': 100,
  'timestamp': 1516578139.736879,
  'transactions': []},
 {'index': 2,
  'previous_hash': 'a782f8423ac42166c7400f67c7c10664a8ca7497a90eb3206497d3ecbf0f0ad5',
  'proof': 35293,
  'timestamp': 1516578139.955023,
  'transactions': [{'amount': 1,
    'recipient': '05069c3d8d3b490290ac0dbfb0d51b30',
    'sender': '0'}]},
 {'index': 3,
  'previous_hash': '08a3a2bfe75711cd0bada143400ee80f06d18e3f4c432f50df4767379e8ada9a',
  'proof': 35089,
  'timestamp': 1516578576.0335057,
  'transactions': [{'amount': '10', 'recipient': 'alice', 'sender': 'bob'},
   {'amount': '1000000', 'recipient': 'brady', 'sender': 'tom'},
   {'amount': '30', 'recipient': 'smith', 'sender': 'john'},
   {'amount': 1,
    'recipient': '05069c3d8d3b490290ac0dbfb0d51b30',
    'sender': '0'}]},
 {'index': 4,
  'previous_hash': '9e8113610bd0abeadd166872d5618b89636c97294f162556e6bb564bab06212e',
  'proof': 119678,
  'timestamp': 1516578595.315006,
  'transactions': [{'amount'